# Оптимизировать производство



In [1]:
import pandas as pd
import pulp as plp

In [2]:
input_file = "input_data.xlsx"
output_file = "result.xlsx"

In [3]:
sheetname = "TEST"
df_orders = pd.read_excel(input_file, sheetname)
df_orders

,MatrixName,Order
0,Орион - 600,150
1,Орион - 700,100
2,Орион - 800,200
3,Весна - 600,218
4,Весна - 700,350
5,Весна - 800,820
6,Сафари,292
7,Сафари 2,352
8,Сатурн ПГ,42
9,Сатурн ПО,584


- NORDERS
- NLOT - кол-во мест для матриц пресса
- NSTEPS - оценка на максимальное кол-во шагов, чтобы выполнить все заказы (ПЕРЕСМОТРЕТЬ!!! Может неправильно!)
- TCHANGE
- THEAT
- TMAX - оценка на максимальное время выполнения любого заказа

In [4]:
df_orders = df_orders[:2]
df_orders

,MatrixName,Order
0,Орион - 600,150
1,Орион - 700,100


In [5]:
NORDERS = df_orders.shape[0]
NLOT = 1
#NSTEPS = int(df_orders.Order.sum()/df_orders.Order.mean()/(NLOT-1)) + 1
NSTEPS = 2 #int((NORDERS)/NLOT)+1

TCHANGE = 20
THEAT = 40
TMAX = 1000#int(df_orders.Order.sum()/NLOT)

In [6]:
NORDERS, NLOT, NSTEPS, TMAX 

(2, 1, 2, 1000)

In [7]:
ORDERS = set(df_orders.MatrixName.values)
ORDERS

{'Орион - 600', 'Орион - 700'}

In [8]:
def convert_dataframe_to_dict(dataframe, key_columns, value_column):
    return (
        dataframe.loc[:, key_columns + [value_column]]
        .set_index(key_columns)
        .to_dict()[value_column]
    )

In [9]:
DURATION = convert_dataframe_to_dict(df_orders, ["MatrixName"], "Order")
DURATION

{'Орион - 600': 150, 'Орион - 700': 100}

## Задача

$a_{j, k, m}$ - заказ j выполнился на шаге k в лоте m

$t_{j, m}$ - время начала выполнения заказа j, в лоте m

$n_j$ - количество отановок пока выполнлся заказ j

In [10]:
prob = plp.LpProblem("ProductionPlan", plp.LpMinimize)

In [11]:
a = plp.LpVariable.dicts(
            "a",
            [
                (j, k, m) for j in ORDERS for k in range(NSTEPS) for m in range(NLOT)
            ],
            cat=plp.LpBinary,
        )

In [12]:
a

{('Орион - 700', 0, 0): a_('Орион___700',_0,_0),
 ('Орион - 700', 1, 0): a_('Орион___700',_1,_0),
 ('Орион - 600', 0, 0): a_('Орион___600',_0,_0),
 ('Орион - 600', 1, 0): a_('Орион___600',_1,_0)}

In [13]:
t = plp.LpVariable.dicts(
            "t",
            [(j, k, m) for j in ORDERS for k in range(NSTEPS) for m in range(NLOT)],
            lowBound=0
        )

In [14]:
n = plp.LpVariable.dicts(
            "n",
            [(j, k) for j in ORDERS for k in range(NSTEPS)], cat=plp.LpInteger, lowBound=0, upBound=NSTEPS-1)

In [15]:
# objective
prob += plp.lpSum(t[j, k, m] + (DURATION[j] + THEAT) * a[j, k, m] for j in ORDERS for k in range(NSTEPS) for m in range(NLOT)) \
     + plp.lpSum(n[j, k]*TCHANGE for j in ORDERS for k in range(NSTEPS))

In [16]:
for j in ORDERS:
    prob += plp.lpSum(a[j, k, m] for k in range(NSTEPS) for m in range(NLOT)) == 1, f"job:{j}"

In [17]:
#for k in range(NSTEPS):
#    for m in range(NLOT):
#        prob += plp.lpSum(a[j, k, m] for j in ORDERS) == 1, f"step:{k}, lot:{m}"

In [18]:
# wrong
#for j in ORDERS:
#    for k in range(NSTEPS):
#        prob += plp.lpSum(a[j, k, m] for m in range(NLOT)) <= 1

In [19]:
for m in range(NLOT):
    for k in range(NSTEPS - 1):
        for j1 in ORDERS:
            for j2 in ORDERS:
                if j1 == j2:
                    continue
                prob += t[j1, k, m] + (DURATION[j1] + THEAT) + TCHANGE * n[j1, k] - t[j2, k+1, m] <= TMAX * (1 - a[j2, k+1, m])
                prob += t[j2, k, m] + (DURATION[j2] + THEAT) + TCHANGE * n[j2, k] - t[j1, k+1, m] <= TMAX * (1 - a[j1, k+1, m])

In [20]:
#for m in range(NLOT):
#    for k in range(NSTEPS):
#        for j in ORDERS:
#            prob += t[j, k, m] <= (TMAX - DURATION[j]) * a[j, k, m]

In [21]:
prob.solve(), plp.value(prob.objective)

(1, 330.0)

In [22]:
plp.LpStatus[prob.status]

'Optimal'

In [23]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

a_('Орион___600',_0,_0) = 1.0
a_('Орион___600',_1,_0) = 0.0
a_('Орион___700',_0,_0) = 1.0
a_('Орион___700',_1,_0) = 0.0
n_('Орион___600',_0) = 0.0
n_('Орион___600',_1) = 0.0
n_('Орион___700',_0) = 0.0
n_('Орион___700',_1) = 0.0
t_('Орион___600',_0,_0) = 0.0
t_('Орион___600',_1,_0) = 0.0
t_('Орион___700',_0,_0) = 0.0
t_('Орион___700',_1,_0) = 0.0


In [24]:
prob

ProductionPlan:
MINIMIZE
190*a_('Орион___600',_0,_0) + 190*a_('Орион___600',_1,_0) + 140*a_('Орион___700',_0,_0) + 140*a_('Орион___700',_1,_0) + 20*n_('Орион___600',_0) + 20*n_('Орион___600',_1) + 20*n_('Орион___700',_0) + 20*n_('Орион___700',_1) + 1*t_('Орион___600',_0,_0) + 1*t_('Орион___600',_1,_0) + 1*t_('Орион___700',_0,_0) + 1*t_('Орион___700',_1,_0) + 0
SUBJECT TO
job:Орион___700: a_('Орион___700',_0,_0) + a_('Орион___700',_1,_0) = 1

job:Орион___600: a_('Орион___600',_0,_0) + a_('Орион___600',_1,_0) = 1

_C1: 1000 a_('Орион___600',_1,_0) + 20 n_('Орион___700',_0)
 - t_('Орион___600',_1,_0) + t_('Орион___700',_0,_0) <= 860

_C2: 1000 a_('Орион___700',_1,_0) + 20 n_('Орион___600',_0)
 + t_('Орион___600',_0,_0) - t_('Орион___700',_1,_0) <= 810

_C3: 1000 a_('Орион___700',_1,_0) + 20 n_('Орион___600',_0)
 + t_('Орион___600',_0,_0) - t_('Орион___700',_1,_0) <= 810

_C4: 1000 a_('Орион___600',_1,_0) + 20 n_('Орион___700',_0)
 - t_('Орион___600',_1,_0) + t_('Орион___700',_0,_0) <= 860